In [ ]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
import requests

In [ ]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b", torch_dtype=torch.float16, low_cpu_mem_usage=True)
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
prompt = "What is unusual about this image?"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=1,
)
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(generated_text)

# Count

In [ ]:
import torch
from PIL import Image
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from collections import defaultdict
import pandas as pd
import re

# Load the InstructBLIP model and processor
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-vicuna-7b",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to("cuda:0")

In [ ]:
# Load the dataset
data = pd.read_csv("[without images]1_correct_validation.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})


#Get the correct count
correct_cnt = 0
total_questions = 0   

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories
    
    total_questions += 1
    
    # Load the corresponding image
    image_path = f"1_correct_validation_images/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        correct_cnt += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']}")

# Calculate Accuracy
print(f"Total Correct:  {correct_cnt}")
print(f"Total Questions:  {total_questions}")
print(f"Accuracy: {(correct_cnt/total_questions) * 100} % ")

In [ ]:
# Load the dataset
data = pd.read_csv("[without images]1_correct_validation.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})


#Get the correct count
correct_cnt = 0
total_questions = 0   

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories
    
    total_questions += 1
    
    # Load the corresponding image
    image_path = f"1_correct_validation_images/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        correct_cnt += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']}")

# Calculate Accuracy
print(f"Total Correct:  {correct_cnt}")
print(f"Total Questions:  {total_questions}")
print(f"Accuracy: {(correct_cnt/total_questions) * 100} % ")

In [ ]:
# Load the dataset
data = pd.read_csv("[without images]1_correct_validation.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})


#Get the correct count
correct_cnt = 0
total_questions = 0   

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories
    
    total_questions += 1
    
    # Load the corresponding image
    image_path = f"1_correct_validation_images/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        correct_cnt += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']}")

# Calculate Accuracy
print(f"Total Correct:  {correct_cnt}")
print(f"Total Questions:  {total_questions}")
print(f"Accuracy: {(correct_cnt/total_questions) * 100} % ")

In [ ]:
# Load the dataset
data = pd.read_csv("[without images]1_correct_validation.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})


#Get the correct count
correct_cnt = 0
total_questions = 0   

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "vcr":
        continue  # Skip non-count categories
    
    total_questions += 1
    
    # Load the corresponding image
    image_path = f"1_correct_validation_images/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        correct_cnt += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']}")

# Calculate Accuracy
print(f"Total Correct:  {correct_cnt}")
print(f"Total Questions:  {total_questions}")
print(f"Accuracy: {(correct_cnt/total_questions) * 100} % ")